In [42]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from goatools.obo_parser import GODag
from goatools.anno.genetogo_reader import Gene2GoReader
from scipy.stats import chi2_contingency, fisher_exact
from statsmodels.stats.multitest import multipletests

In [4]:
from dataManager import DataManager, LogManager

In [5]:
# Prepare the logger
logger = LogManager(is_active=True)

# Initialize the DataManager class and import all the data
DATA_PATH = "../../preparation/codice/"
HPO2GENES_PATH = f"{DATA_PATH}phenotype_to_genes.txt"
HPO2GENES_PROVA_PATH = f"{DATA_PATH}HPO2Genes_head.csv"
GO_ONTOLOGY_PATH = f"{DATA_PATH}go-basic.obo" 
GENE2GO_PATH = f"{DATA_PATH}gene2go"  # Path to the downloaded gene2go file

data_manager = DataManager()

print(DATA_PATH)

../../preparation/codice/


In [6]:
logger.log("Importing HPO2Genes file...")
data_manager.importHPO2GeneFile(HPO2GENES_PATH, L_bound = 50, R_bound = 100)
logger.log("HPO2Genes file imported.")
print(data_manager.hpo_head())

Importing HPO2Genes file...
HPO2Genes file imported.
hpo_id        HP:0000003  HP:0000011  HP:0000012  HP:0000013  HP:0000022  \
ncbi_gene_id                                                               
16                     0           0           0           0           0   
18                     0           0           0           0           0   
19                     0           0           0           0           0   
20                     0           0           0           0           0   
21                     0           0           0           0           0   

hpo_id        HP:0000046  HP:0000056  HP:0000058  HP:0000066  HP:0000072  ...  \
ncbi_gene_id                                                              ...   
16                     0           0           0           0           0  ...   
18                     0           0           0           0           0  ...   
19                     0           0           0           0           0  ...   
20       

In [8]:
humanTaxID = 9606
GO_taxonomies = [humanTaxID]
logger.log("\nImporting GO2Genes file...")
data_manager.importGO2GeneFile(go_ontology_path=GO_ONTOLOGY_PATH, gene2go_path=GENE2GO_PATH, taxids = GO_taxonomies)
logger.log("GO2Genes file imported.")
print(data_manager.go_head())


Importing GO2Genes file...
../../preparation/codice/go-basic.obo: fmt(1.2) rel(2024-10-27) 44,017 Terms
**NOTE: DEFAULT TAXID STORED FROM gene2go IS 9606 (human)

HMS:0:03:19.954839 362,883 annotations, 20,819 genes, 18,767 GOs, 1 taxids READ: ../../preparation/codice/gene2go 
20785 IDs in loaded association branch, biological_process
GO2Genes file imported.
GO    GO:0000002  GO:0000009  GO:0000012  GO:0000014  GO:0000015  GO:0000016  \
Gene                                                                           
1              0           0           0           0           0           0   
2              0           0           0           0           0           0   
9              0           0           0           0           0           0   
10             0           0           0           0           0           0   
12             0           0           0           0           0           0   

GO    GO:0000017  GO:0000018  GO:0000019  GO:0000022  ...  GO:2001288  \
Gene

In [7]:
data_manager.hpo_head()

hpo_id,HP:0000003,HP:0000011,HP:0000012,HP:0000013,HP:0000022,HP:0000046,HP:0000056,HP:0000058,HP:0000066,HP:0000072,...,HP:3000036,HP:5200005,HP:5200010,HP:5200017,HP:5200018,HP:5200029,HP:5200123,HP:5201016,HP:6000231,HP:6000531
ncbi_gene_id,,,,,,,,,,,,,,,,,,,,,
16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Statistical Test (Chi2 or Fisher)

In [ ]:
def compute_p_value(go_series, hpo_series, method = "chi2"):
    # Confusion matrix components
    both = ((go_series == 1) & (hpo_series == 1)).sum()
    only_go = ((go_series == 1) & (hpo_series == 0)).sum()
    only_hpo = ((go_series == 0) & (hpo_series == 1)).sum()
    neither = ((go_series == 0) & (hpo_series == 0)).sum()
    
    # Contingency table with Laplace Smoothing to prevent some cells to have frequency 0
    contingency_table = [[both + 1, only_hpo + 1],
                         [only_go + 1, neither + 1]]
    
    p_value = 1
    # Chi-Square test (or switch to Fisher's Exact Test if preferred)
    if method.lower() == "chi2":
        _, p_value, _, _ = chi2_contingency(contingency_table)
    elif method.lower() == "fisher":
        _, p_value = fisher_exact(contingency_table)
        
    return p_value

In [19]:
# Randomly pick one column from the DataFrame
random_hpo_column = random.choice(data_manager.hpo_gene_data.columns)
print(random_hpo_column)
hpo_series = data_manager.hpo_gene_data[random_hpo_column]
hpo_series

HP:0001199


ncbi_gene_id
16           0
18           0
19           0
20           0
21           0
            ..
101101692    0
101928376    0
101929726    0
105371045    0
120766137    0
Name: HP:0001199, Length: 4723, dtype: Sparse[int32, 0]

In [50]:
sample_go_columns = random.sample(list(data_manager.go_gene_data.columns), 1000)
sample_go_columns[:10]

['GO:0043227',
 'GO:0050730',
 'GO:0180034',
 'GO:0008143',
 'GO:0045586',
 'GO:0097361',
 'GO:0034188',
 'GO:0071036',
 'GO:0002326',
 'GO:0005246']

In [51]:
sample_df = data_manager.get_dataset(go_list=sample_go_columns, hpo_list=random_hpo_column)
print(sample_df.shape)
sample_df.head()

(4685, 1001)


,HP:0001199,GO:0043227,GO:0050730,GO:0180034,GO:0008143,GO:0045586,GO:0097361,GO:0034188,GO:0071036,GO:0002326,...,GO:0007155,GO:0048006,GO:0004515,GO:0047631,GO:0004502,GO:0002436,GO:0015655,GO:0140719,GO:2000860,GO:0009008
16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
# Apply the function across GO columns (sampled)
p_values = sample_df[sample_go_columns].apply(lambda go_col: compute_p_value(go_col, sample_df[random_hpo_column], method="fisher"))


results_df = pd.DataFrame({'GO_Term': sample_go_columns, 'P_Value': p_values.values})

# Filter significant GO terms
significant_go_terms = results_df[results_df['P_Value'] < 0.05]


significant_go_terms.head()

,GO_Term,P_Value
2,GO:0180034,0.045371
4,GO:0045586,0.030478
5,GO:0097361,0.030478
7,GO:0071036,0.030478
13,GO:0000322,0.045371


In [ ]:
# multiple testing correction
#  method='bonferroni' for Bonferroni correction
#  method='fdr_bh' for Benjamini-Hochberg
corrected_results = multipletests(results_df['P_Value'], method='fdr_bh')  # Benjamini-Hochberg

# print(corrected_results)
results_df['Adjusted_P_Value'] = corrected_results[1]  # Corrected p-values
results_df['Significant'] = corrected_results[0]       # True/False for significance

significant_results_corrected = results_df[results_df['Significant']]

print(f"HPO term: {random_hpo_column}")
print(significant_results_corrected)

HPO term: HP:0001199
        GO_Term       P_Value  Adjusted_P_Value  Significant
721  GO:0000027  3.396904e-05      3.396904e-02         True
882  GO:0002181  3.265716e-36      3.265716e-33         True
905  GO:0001833  1.555300e-05      1.555300e-02         True


In [62]:
print(results_df.shape)
print(significant_go_terms.shape)
print(significant_results_corrected.shape)
print(significant_results_corrected["GO_Term"].to_list())

(1000, 4)
(553, 2)
(3, 4)
['GO:0000027', 'GO:0002181', 'GO:0001833']


### Now we can obtain an association binary matrix which contains the HPO term and its significative GO terms

In [64]:
corrected_go_list = significant_results_corrected["GO_Term"].to_list()
corrected_df = data_manager.get_dataset(go_list=corrected_go_list, hpo_list=random_hpo_column)
print(f"Dimensions: {corrected_df.shape}")
corrected_df.head()

Dimensions: (4685, 4)


,HP:0001199,GO:0000027,GO:0002181,GO:0001833
16,0,0,0,0
18,0,0,0,0
19,0,0,0,0
20,0,0,0,0
21,0,0,0,0
